In [2]:
import os
import sys
import numpy as np
import pandas as pd

clone_base = '/Users/mojo/git_clone/SAM_BAM/'
wdl_directory = os.path.join(clone_base,'MayomicsVC/src/wdl')
os.listdir(wdl_directory)

['SomaticVC',
 'DeliveryOfSomaticVC',
 'GermlineMasterWorkflow.wdl',
 'config_vars.ipynb',
 'Alignment',
 '.ipynb_checkpoints',
 'SomaticMasterWorkflow.wdl',
 'DeliveryOfHaplotyperVC',
 'DeliveryOfAlignment',
 'HaplotyperVC']

In [3]:
# Get the list of all tpyes in the tasks section of the wdl files:
skip_line_chars = ['#', '<', '>']
switch_ON_words = ['task']
switch_OFF_words = ['command', 'runtime', 'output']
add_words_ON = False

first_words = []
for this_dir, dirs, files in os.walk(wdl_directory):
    for file in files:
        if file[-4:] == '.wdl':
            full_filename = os.path.join(this_dir, file)
            with open(full_filename, 'r') as fh:
                lines = fh.readlines()
            for line in lines:
                l = line.strip()
                if len(l) > 0 and not l[0] in skip_line_chars:
                    first_word = l.split()[0]
                    if first_word in switch_OFF_words:
                        add_words_ON = False
                    if first_word in switch_ON_words:
                        add_words_ON = True
                    if add_words_ON == True:
                        first_words.append(first_word)

first_words = sorted(list(set(first_words)))
print('There are %i unique types in all the wdl files:'%(len(first_words)))
for wurd in first_words:
    print('\t',wurd)

There are 6 unique types in all the wdl files:
	 Array[File]
	 Boolean
	 File
	 File?
	 String
	 task


In [4]:
# Get the complete config.txt file:
config_vars_dict = {}
for this_dir, dirs, files in os.walk(wdl_directory):
    for file in files:
        if file[-4:] == '.wdl' and file[0] != '.':
            full_filename = os.path.join(this_dir, file)
            with open(full_filename, 'r') as fh:
                lines = fh.readlines()
            for line in lines:
                l = line.strip()
                if len(l) > 0 and not l[0] in skip_line_chars:
                    line_words_list = l.split()
                    first_word = line_words_list[0]
                    if first_word in first_words:
                        second_word = line_words_list[1]
                        if not second_word in config_vars_dict.keys():
                            config_vars_dict[second_word] = '"' + first_word + '"'
                        elif first_word != config_vars_dict[second_word][1:-1]:
                            bugger = config_vars_dict[second_word][:-1]
                            config_vars_dict[second_word] = bugger + '_' + first_word  + '"'
                            
print('found %i variables'%(len(config_vars_dict)))
for k, v in config_vars_dict.items():
    print('%30s: %s'%(k,v))

found 130 variables
                      Trimming: "Boolean"
                MarkDuplicates: "Boolean"
                          Bqsr: "Boolean"
                          Vqsr: "Boolean"
        DeliverAlignOutputBams: "File"
        DeliverAlignOutputBais: "File"
          DeliverHaplotyperVcf: "File"
       DeliverHaplotyperVcfIdx: "File"
                   strelkaTask: "task"
                     TumorBams: "File"
                     TumorBais: "File"
                    NormalBams: "File"
                    NormalBais: "File"
                           Ref: "File"
                        RefFai: "File"
                    SampleName: "String"
     StrelkaExtraOptionsString: "String"
                       Strelka: "String"
                StrelkaThreads: "String"
                  BashPreamble: "File"
           BashSharedFunctions: "File"
                 StrelkaScript: "File"
             StrelkaEnvProfile: "File"
                     DebugMode: "String"
                     O